In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Regg").getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.csv("tips.csv", header=True, inferSchema=True)

In [5]:
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [7]:
### handling Categoiral Features
from pyspark.ml.feature import StringIndexer

In [8]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [9]:
indexer = StringIndexer(inputCol='sex', outputCol= 'sex_indexed')
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [10]:
df.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [11]:
indexer = StringIndexer(inputCols= ['sex','smoker', 'day', 'time'], outputCols= ['sex_indexed','smoker_index', 'day_index', 'time_index'])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+------------+---------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_index|day_index|time_index|
+----------+----+------+------+---+------+----+-----------+------------+---------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|         0.0|      1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|         0.0|      1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|         0.0|      1.0|       0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|         0.0|      1.0|       0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|         0.0|      1.0|       0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0| 

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
Independent_Feature = VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_index', 'day_index', 'time_index'], outputCol='Independent Feature')
Output_feature = Independent_Feature.transform(df_r)
Output_feature.show()

+----------+----+------+------+---+------+----+-----------+------------+---------+----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_index|day_index|time_index| Independent Feature|
+----------+----+------+------+---+------+----+-----------+------------+---------+----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|         0.0|      1.0|       0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|         0.0|      1.0|       0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|         0.0|      1.0|       0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|         0.0|      1.

In [14]:
Output_feature.select("Independent Feature").show()

+--------------------+
| Independent Feature|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [15]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_index',
 'day_index',
 'time_index']

In [17]:
Output_feature.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_index',
 'day_index',
 'time_index',
 'Independent Feature']

In [18]:
finalized_data=Output_feature.select("Independent Feature","total_bill")

In [20]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [24]:
regressor.coefficients

DenseVector([3.2813, 3.2973, -0.5636, 1.5323, -0.2319, -1.376])

In [25]:
regressor.intercept

1.6190941907401262

In [21]:
pred_results=regressor.evaluate(test_data)

In [22]:
pred_results.predictions.show()

+--------------------+----------+------------------+
| Independent Feature|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.75,2.0])|     17.82|13.956064267122832|
|(6,[0,1],[2.24,3.0])|     16.04| 18.86123890031422|
|(6,[0,1],[2.72,2.0])|     13.28|17.138950222577286|
| (6,[0,1],[3.0,2.0])|      14.0| 18.05772142621362|
|(6,[0,1],[3.39,2.0])|     11.61|19.337438459849945|
| (6,[0,1],[3.6,3.0])|     24.06|   23.323841889405|
| (6,[0,1],[5.0,3.0])|     31.27|27.917697907586682|
|[1.01,2.0,1.0,0.0...|     16.99|10.732300027704591|
|[1.5,2.0,0.0,1.0,...|     11.59|14.668054005950593|
|[1.64,2.0,0.0,1.0...|     15.36|15.127439607768759|
|[1.71,2.0,0.0,0.0...|     10.27|13.592877890395343|
|[2.0,2.0,0.0,0.0,...|      8.77|14.544462351304407|
|[2.0,2.0,0.0,0.0,...|     13.13|14.544462351304407|
|[2.0,2.0,0.0,0.0,...|     13.03|12.936565287554552|
|[2.0,2.0,0.0,0.0,...|     15.95|12.936565287554552|
|[2.0,2.0,0.0,1.0,...|     22.67|16.3087168695

In [23]:
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError


(0.5089460105392632, 4.227609849624066, 29.515861320403513)